In [1]:
# Standard libraries
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
plt.rcParams['figure.dpi'] = 300
plt.rcParams['text.usetex'] = False
import random
import csv
import pandas as pd
# import h5py
# Scikit learn libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# PyTorch libraries
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms import ToTensor, Normalize 
# Own scripts:
%load_ext autoreload
%autoreload 2
import physics
import data
import nnc2p
from collections import OrderedDict
# from nnc2p import NeuralNetwork # our own architecture
# Get dirs
import os
cwd = os.getcwd()# "Code" folder
master_dir = os.path.abspath(os.path.join(cwd, ".."))

In [2]:
class Net(nn.Module):
    """
    Implements a simple feedforward neural network.
    """
    def __init__(self, nb_of_inputs: int = 3, nb_of_outputs: int = 1, h: list = [600, 200], 
                 reg: bool = False, activation_function = torch.nn.Sigmoid, output_bias=True) -> None:
        """
        Initialize the neural network class.
        """
        # Call the super constructor first
        super(Net, self).__init__()
        
        # For convenience, save the sizes of the hidden layers as fields as well
        self.h = h
        # Add visible layers as well: input is 3D and output is 1D
        self.h_augmented = [nb_of_inputs] + h + [nb_of_outputs]

        # Add field to specify whether or not we do regularization
        self.regularization = reg

        # Define the layers:
        for i in range(len(self.h_augmented)-1):
            # For the final layer, do/do not use a bias term (user choice)
            if i == len(self.h_augmented)-2:
                setattr(self, f"linear{i+1}", nn.Linear(self.h_augmented[i], self.h_augmented[i+1], bias=output_bias))
            else:
                setattr(self, f"linear{i+1}", nn.Linear(self.h_augmented[i], self.h_augmented[i+1]))
                setattr(self, f"activation{i+1}", activation_function())

    def forward(self, x):
        """
        Computes a forward step given the input x.
        :param x: Input for the neural network.
        :return: x: Output neural network
        """

        for i, module in enumerate(self.modules()):
            # The first module is the whole NNC2P object, continue
            if i == 0:
                continue
            x = module(x)

        return x

# Read in data

In [3]:
# Give the names of the input vars (features) and output vars (labels)
in_vars = ["D", "S", "tau"]
out_vars = ["p"]
# For normalization, use sklearn's StandardScaler -- give None for no normalization
scaler = StandardScaler()
# Read the sampled data as pandas dataframes
train_df = pd.read_csv(os.path.join(master_dir, "Data/ideal_gas_c2p_train_data.csv"))
test_df  = pd.read_csv(os.path.join(master_dir, "Data/ideal_gas_c2p_test_data.csv"))
# Convert to PyTorch Datasets as we defined them
train_dataset = data.CustomDataset(train_df, feature_names = in_vars, label_names = out_vars, normalization_function = None)  # scaler.fit_transform 
test_dataset  = data.CustomDataset(test_df, feature_names = in_vars, label_names = out_vars, normalization_function = None)  # scaler.transform
# Then create dataloaders, with batch size 32, from datasets
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader  = DataLoader(test_dataset, batch_size=32)

Get the mu for output:

In [42]:
for df in [train_df, test_df]:
    W = np.sqrt(1/(1-df["v"]**2))
    h = 1 + df["eps"] + (df["p"]/df["rho"])
    
    df["mu"] = 1/(W*h)

In [43]:
df

,rho,eps,v,p,D,S,tau,mu
0,9.836323,1.962039,0.266066,12.866164,10.204131,12.026585,22.131297,0.225747
1,5.203042,1.522893,0.669504,5.282451,7.004551,22.337426,21.077156,0.209942
2,8.591172,0.739712,0.441473,4.236664,9.574739,10.518784,10.015183,0.401851
3,0.626353,1.392560,0.585310,0.581489,0.772503,1.851943,1.810045,0.244151
4,10.087245,0.222690,0.637219,1.497551,13.088716,14.838680,8.700349,0.562070
...,...,...,...,...,...,...,...,...
9995,2.354663,1.431949,0.609409,2.247838,2.969850,7.730559,7.467644,0.234117
9996,0.646230,0.874169,0.072389,0.376610,0.647930,0.115541,0.571578,0.405941
9997,0.854028,1.505251,0.331675,0.857018,0.905272,1.116739,1.604678,0.268869
9998,6.153600,1.333735,0.350154,5.471517,6.569503,7.914822,10.562783,0.290637


Convert to appropriate output

In [44]:
# Give the names of the input vars (features) and output vars (labels)
in_vars = ["D", "S", "tau"]
out_vars = ["mu"]
# For normalization, use sklearn's StandardScaler -- give None for no normalization
scaler = StandardScaler()
# Convert to PyTorch Datasets as we defined them
train_dataset = data.CustomDataset(train_df, feature_names = in_vars, label_names = out_vars, normalization_function = None)  # scaler.fit_transform 
test_dataset  = data.CustomDataset(test_df, feature_names = in_vars, label_names = out_vars, normalization_function = None)  # scaler.transform
# Then create dataloaders, with batch size 32, from datasets
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader  = DataLoader(test_dataset, batch_size=32)

# Train network

In [127]:
model = Net(h=[10, 10], output_bias=True).float()
device = "cpu"
model

Net(
  (linear1): Linear(in_features=3, out_features=10, bias=True)
  (activation1): Sigmoid()
  (linear2): Linear(in_features=10, out_features=10, bias=True)
  (activation2): Sigmoid()
  (linear3): Linear(in_features=10, out_features=1, bias=True)
)

In [128]:
nnc2p.count_parameters(model)

161

In [129]:
trainer = nnc2p.Trainer(model, 1e-2, train_dataloader=train_dataloader, test_dataloader=test_dataloader)

In [130]:
trainer.train()

Training the model for 500 epochs.

 Epoch 0 
 --------------
Train loss: 5.16E-04
Test  loss: 4.20E-04

 Epoch 1 
 --------------
Train loss: 2.99E-04
Test  loss: 2.36E-04

 Epoch 2 
 --------------
Train loss: 2.45E-04
Test  loss: 2.01E-04

 Epoch 3 
 --------------
Train loss: 1.94E-04
Test  loss: 1.63E-04

 Epoch 4 
 --------------
Train loss: 1.54E-04
Test  loss: 1.27E-04

 Epoch 5 
 --------------
Train loss: 1.29E-04
Test  loss: 1.03E-04

 Epoch 6 
 --------------
Train loss: 1.15E-04
Test  loss: 9.12E-05

 Epoch 7 
 --------------
Train loss: 1.05E-04
Test  loss: 8.25E-05

 Epoch 8 
 --------------
Train loss: 9.76E-05
Test  loss: 7.65E-05

 Epoch 9 
 --------------
Train loss: 9.32E-05
Test  loss: 7.30E-05

 Epoch 10 
 --------------
Train loss: 8.75E-05
Test  loss: 6.81E-05

 Epoch 11 
 --------------
Train loss: 8.12E-05
Test  loss: 6.16E-05

 Epoch 12 
 --------------
Train loss: 7.73E-05
Test  loss: 5.76E-05

 Epoch 13 
 --------------
Train loss: 7.55E-05
Test  loss: 5.60

Train loss: 2.50E-05
Test  loss: 2.49E-05

 Epoch 117 
 --------------
Train loss: 2.49E-05
Test  loss: 2.48E-05

 Epoch 118 
 --------------
Train loss: 2.49E-05
Test  loss: 2.48E-05

 Epoch 119 
 --------------
Train loss: 2.48E-05
Test  loss: 2.47E-05

 Epoch 120 
 --------------


KeyboardInterrupt: 

In [131]:
torch.save(model, "../Models/nn_assist_10_10_sigmoid.pt")

## Test its performance

In [132]:
model = torch.load("../Models/nn_assist_10_10_sigmoid.pt")

In [133]:
x = np.transpose(np.array([train_df["D"].values, train_df["S"].values, train_df["tau"].values]))
y = train_df["mu"].values

# Predict
with torch.no_grad():
    x       = torch.from_numpy(x).float()
    yhat = model(x)
    yhat = yhat.numpy()

## Take for a spin

In [134]:
x = torch.Tensor([0.90437025614154176, 0.70543713884138814, 1.6228154380654374]).float()
x = torch.Tensor([1.1142615696244609, 0.52933694956096999, 1.7985517610787460]).float()

with torch.no_grad():
    y = model(x)
    y = y.item()
print(y)
print([0.9*y, 1.1*y])

0.2722380757331848
[0.24501426815986635, 0.2994618833065033]


In [137]:
x = torch.Tensor([1, 1, 1]).float()

with torch.no_grad():
    y = model(x)
    y = y.item()
print(y)
print([0.9*y, 1.1*y])

0.41238653659820557
[0.371147882938385, 0.45362519025802617]


In [138]:
model.state_dict()["linear3.bias"]

tensor([-0.1158])

# Analyze the results

In [165]:
simulation = "Shocktube"
simulation_dir = os.path.join("../Data/NN assist", simulation)
print(os.path.abspath(simulation_dir))

D:\Coding\master-thesis-AI\Data\NN assist\Shocktube


In [166]:
path = os.path.join(simulation_dir, "counts_old.dat")
counts_old = np.loadtxt(path)
path = os.path.join(simulation_dir, "counts_new.dat")
counts_new = np.loadtxt(path)

In [167]:
print(f"Without assist: {np.mean(counts_old)} +- {np.std(counts_old)}")
print(f"With       assist: {np.mean(counts_new)} +- {np.std(counts_new)}")

Without assist: 3.8004432965140773 +- 1.9823067885472485
With       assist: 3.171163744939642 +- 1.579099339667568
